In [ ]:
import json
import math
import networkx as nx
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import distfit
import os

from generator_v2 import Generator

In [ ]:
alt.data_transformers.enable("vegafusion")

In [ ]:
COLOR = "#99d8c9"

In [ ]:
alt.__version__

In [ ]:
# define the theme by returning the dictionary of configurations

font = "monospace"
# Axes
axisColor = "#000000"
gridColor = "#DEDDDD"
# Colors
main_palette = ["#1696d2", 
                "#d2d2d2",
                "#000000", 
                "#fdbf11", 
                "#ec008b", 
                "#55b748", 
                "#5c5859", 
                "#db2b27", 
               ]
sequential_palette = ["black", 
                      "#a2d4ec", 
                      "#73bfe2", 
                      "#46abdb", 
                      "#1696d2", 
                      "#12719e", 
                     ]


# markColor = "#8856a7"
# markColor = "#6a51a3"
markColor = "#9ebcda"

# Define custom color schemes for ordinal and categorical data
categorical_color_scheme = ['red', 'green', 'blue', 'purple', 'orange']
ordinal_color_scheme = ["#fcfbfd",
"#efedf5",
"#dadaeb",
"#bcbddc",
"#9e9ac8",
"#807dba",
"#6a51a3",
"#4a1486"]


def simple_theme():
    return {
        'config': {
            'view': {
                'height': 300,
                'width': 300,
            },
            "title": {
                    "fontSize": 18,
                    "font": font,
                    "anchor": "start", # equivalent of left-aligned.
                    "fontColor": "#000000"
                },
            "facet": {
                "labelFontSize": 16,
                "titleFontSize": 20  # Adjust the fontsize as needed
            },
            "axisX": {
                    "labelFont": font,
                    "labelFontSize": 16,
                    "titleFontSize": 20,
                    "grid": False,
                },
                "axisY": {
                    "domain": False,
                    "grid": False,
                    "labelFont": font,
                    "labelFontSize": 16,
                    "titleFontSize": 20,
                },
            'mark': {
                'color': markColor,
                'fill': markColor
            },
            "line": {
                "fill": None,
               "stroke": "black",
           },
            # "range": {
            #         "category": categorical_color_scheme,
            #         "ordinal": ordinal_color_scheme,
            #         "diverging": sequential_palette,
            #     },
            "scale": {"color": {"scheme": "blue"}},
            "legend": {

                "titleFontSize": 20,
                "labelFontSize": 16
            }            
        }
    }

In [ ]:
# alt.themes.register("theme", theme)
alt.themes.register("theme", simple_theme)
alt.themes.enable("theme")

In [ ]:
# Set the ggplot2 theme
#alt.themes.enable("ggplot2")

In [ ]:
alt.themes

In [ ]:
# Essayer avec 1000 pour voir max(n1, n2) / n1 + n2"
N = 1000
# N_edges = N // 2
# Tester plusierus valeurs d'aretes (50, 100, 200), pour la fraction des hyperaretes
N_edges = 200

# Last value is the one used for most simulations
# N_EDGES = [50, 100, 200, 500, 300]
# N_EDGES = [300]t
# N_EDGES = [100, 200, 300, 500]
Ns = [200, 300, 500, 700, 1000, 2000]


N_coms = 4
sampling_strat = "weighted"
# sampling_strat = "max"
p = 30/N
q = 3/N
# p = 0.1

In [ ]:
community_array = [0 for x in range(N//2)]  + [1 for x in range(N//2)]

In [ ]:
len(community_array)

# Fraction Dist

In [ ]:
# Faire pareil sur 100/1000 réseaux
# Accumuler la liste des degrées/hsize
# Une value de p/q => plusieurs simulations => faire le fit la dessus

In [ ]:
p_init = 30 / N
q_init = 30 / N

In [ ]:
p_init = 0.1
q_init = 0.1

In [ ]:
q_init

In [ ]:
def fit_function(x, n, p):
    return stats.binom.pmf(x, n, p)

In [ ]:
qs = np.arange(0, 1.01, 0.10)
# qs = np.append(qs, [0.95, 0.99])
qs = np.append(qs, [0.01, 0.05])

In [ ]:
fits_N = 1000
fits_strat = "weighted"

In [ ]:
q

## SIMULATIONS HERE

In [ ]:
p = p_init
q = q_init

df_degrees = pd.DataFrame(columns=["degree"], dtype=int)
df_hsizes = pd.DataFrame(columns=["hsize"], dtype=int)

df_sim = pd.DataFrame(columns=["sim", "type", "count"], dtype=int)
df_fraction = pd.DataFrame(columns=["sim", "count", "fraction0"], dtype=int)
df_fraction2 = pd.DataFrame(columns=["sim", "value"])

df_fits = pd.DataFrame(columns=["peff", "Neff", "p_hsize_eff", "n_eff_hsize", "p", "q", "q_frac"], dtype=int)

# Same as df_fits but other format so easier altair plots 
df_fits2 = pd.DataFrame()

increment = 0.05
N_sim = int(1 / increment) + 1
q_frac_order = []


# N_graphs = 5
# N_graphs = 50
N_graphs = 100
# N_graphs = 3

q_to_degreefit = {}
q_to_hsizefit = {}

for q_frac in qs:
    q = round(p * q_frac,  4)
    print(p, q)
    
    # q_frac = f"{round(1 - (increment * (i)), 3)}p"
    q_frac_order.append(q_frac) 

    for strat in ["weighted", "max", "min", "frequent"]:
        for n in Ns:
            all_degrees = []
            all_hsizes = []  
    
            community_array = []
            for i in range(N_coms):
                community_array += [i for x in range(n // N_coms)]
            
            for n_graph in range(N_graphs):
                # gen = Generator(N, N_edges, N_coms, p, q, community_array, sampling_strat)
                gen = Generator(n, N_edges, N_coms, p, q, community_array, strat)
                gen.run()
                    
                #     For com distrib of hyperedges
                comp = gen.hyperedges_nmax()
                ginis = gen.ginis()
                
                # for fraction in comp:
                df = pd.DataFrame({"q": [q] * len(comp), "value": comp, "gini": ginis, "N_nodes": [n] * len(comp), "strat": [strat] * len(comp)})
                df_fraction2 = pd.concat([df_fraction2, df])
                
                # for peffectif computation
                if n == fits_N and strat == fits_strat:
                    degrees = dict(gen.degrees()).values()
                    hsizes = dict(gen.hyperedge_sizes()).values()
                    
                    all_degrees = all_degrees + list(degrees)
                    all_hsizes = all_hsizes + list(hsizes)
        

            # DO THE DISTRIBUTIONS AND FIT ONLY FOR THOSE VALUES
            if n == fits_N and strat == fits_strat:
                df = pd.DataFrame({"degree": all_degrees, "q": q_frac, "q / p": q / p, "N_nodes": n})
                df_degrees = pd.concat([df_degrees, df])
                                      
                df = pd.DataFrame({"hsize": all_hsizes, "q": q_frac, "q / p": q / p, "N_nodes": n})
                df_hsizes = pd.concat([df_hsizes, df])
    
                # Distrib of degree
                # Bounds of n and p effectif
                bounds = [(0, N_edges), (0, p)]
                
                dist = stats.binom
                res = stats.fit(dist, all_degrees, bounds)
                
                peff = res.params[1]
                neff = res.params[0]
                
                bounds2 = [(N_edges, N_edges), (0, p)]
                res = stats.fit(dist, all_degrees, bounds2)
                peffNfixed = res.params[1]
                neffNfixed = res.params[0]
                
                bounds3 = [(0, N_edges), (p, p)]
                res = stats.fit(dist, all_degrees, bounds3)
                peffpfixed = res.params[1]
                neffpfixed = res.params[0]
                
                df_notfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff], "Neff": [neff], "bounds": ["not fixed"], "measure": ["degree"]})
                df_pfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peffNfixed], "Neff": [neffNfixed], "bounds": ["n Fixed"], "measure": ["degree"]})
                df_nfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peffpfixed], "Neff": [neffpfixed], "bounds": ["p Fixed"], "measure": ["degree"]})
                df_fits2 = pd.concat([df_fits2, df_notfixed, df_pfixed, df_nfixed])
                
                q_to_degreefit[q] = res
                
                # Distrib of hsizes
                dist = stats.binom
                
                # bounds = [(N / N_coms, N), (q, p)]
                bounds = [(N / N_coms, N), (0, p)]
                res = stats.fit(dist, all_hsizes, bounds)
                peffhsize = res.params[1]
                neff_hsize = res.params[0]
                q_to_hsizefit[q] = res
                
                bounds = [(N / N_coms, N), (p, p)]
                res = stats.fit(dist, all_hsizes, bounds)
                peff_hsize_pfixed = res.params[1]
                neff_hsize_pfixed = res.params[0]
                
                bounds = [(N, N), (0, p)]
                res = stats.fit(dist, all_hsizes, bounds)
                peff_hsize_nfixed = res.params[1]
                neff_hsize_nfixed = res.params[0]
                
                df = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff], "Neff": [neff], "peffpfixed": [peffpfixed], "Neffpfixed": [neffpfixed], "peffnfixed": [peffNfixed], "Neffnfixed": [neffNfixed], "p_hsize_eff": [peffhsize], "n_eff_hsize": [neff_hsize]})
                df_fits = pd.concat([df_fits, df])
                # df_peff = pd.concat([df_peff, df])
                
                df_notfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peffhsize], "Neff": [neff_hsize], "bounds": ["not fixed"], "measure": ["hsize"]})
                df_pfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff_hsize_nfixed], "Neff": [neff_hsize_nfixed], "bounds": ["n Fixed"], "measure": ["hsize"]})
                df_nfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff_hsize_pfixed], "Neff": [neff_hsize_pfixed], "bounds": ["p Fixed"], "measure": ["hsize"]})
                df_fits2 = pd.concat([df_fits2, df_notfixed, df_pfixed, df_nfixed])

In [ ]:
fn = f"he_distrib_{p}p.csv"
if not os.path.exists(fn):
    df_fraction2.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_fraction2 = pd.read_csv(fn)

In [ ]:
df_fraction2["q / p"] = df_fraction2["q"] / p
df_fraction2["N / E"] = df_fraction2["N_nodes"] / N_edges

In [ ]:
df_fraction2 = df_fraction2.drop(["sim"], axis=1)
df_fraction2 = df_fraction2.drop("value", axis=1)
df_fraction2 = df_fraction2.drop("N_nodes", axis=1)

In [ ]:
p

In [ ]:
fn = f"hsize_distrib_{fits_N}_{fits_strat}_{p}p.csv"
if not os.path.exists(fn):
    df_hsizes.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_hsizes = pd.read_csv(fn)

In [ ]:
fn = f"degree_distrib_{fits_N}_{fits_strat}_{p}p.csv"
if not os.path.exists(fn):
    df_degrees.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_degrees = pd.read_csv(fn)

In [ ]:
fn = f"fits1_{fits_N}_{fits_strat}_{p}p.csv"
if not os.path.exists(fn):
    df_fits.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_fits = pd.read_csv(fn)

In [ ]:
fn = f"fits2_{fits_N}_{fits_strat}{p}p.csv"
if not os.path.exists(fn):
    df_fits2.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_fits2 = pd.read_csv(fn)

## Group by

In [ ]:
df_degrees = df_degrees[df_degrees["N_nodes"] == 1000]

In [ ]:
df_degrees.head()

In [ ]:
df_fits

In [ ]:
df_fits = df_fits.round({'q_frac': 3})
df_fits.q_frac

In [ ]:
grouped = df_degrees.groupby('q / p')['degree'].value_counts(normalize=True).reset_index(name='Percentage')

In [ ]:
grouped = grouped.round({'q / p': 2})

In [ ]:
grouped['binomial'] = 0  # Initialize the column

for q in qs:
    q = round(q, 3)
    # print(q)
    df = df_degrees[df_degrees["q"] == f"{q}p"]
    # fit = q_to_degreefit[round(q * p, 3)]
    
    # Calculate the binomial PDF values for a range of x values
    x_values = range(19)  # Assuming a range from 0 to 10

    fit_values = df_fits[df_fits["q_frac"] == q]
    # print(fit_values.Neffpfixed)
    # print(fit_values.Neffpfixed.iloc[0])

    if len(fit_values) > 0:
        for x in x_values:
            binom_pdf = stats.binom.pmf(x, fit_values.Neffpfixed.iloc[0], fit_values.peffpfixed.iloc[0])
            print(binom_pdf, q)
            # grouped.loc[(grouped['degree'] == x) & (grouped['q'] == f"{q}p"), 'binomial'] = binom_pdf
            grouped.loc[(grouped['degree'] == x) & (grouped['q / p'] == q), 'binomial'] = binom_pdf 
    else:
        print("WARNING")

In [ ]:
grouped

In [ ]:
# grouped_selp_deg = grouped[grouped.q.isin(["0.0p", "0.3p", "0.7p", "1.0p"])]
grouped_selp_deg = grouped[grouped["q / p"].isin([0, 0.3, 0.7, 1])]

In [ ]:
grouped_selp_deg

In [ ]:
pdf = alt.Chart(grouped_selp_deg).mark_line(width=1, opacity=0.7, fill=None, color="black").encode(
# pdf = alt.Chart(grouped_selp_deg).mark_line(width=1, opacity=0.7, color="darkgrey", fill=None).encode(
        x=alt.X('degree:Q', title="k", axis=alt.Axis(labelExpr="datum.value % 4 ? null : datum.label"), scale=alt.Scale(domainMin=0)),
        y=alt.Y('binomial:Q', title="P(k)"),
        # color="q"
        # facet=alt.Facet("q:O")    
)

In [ ]:
# distrib = alt.Chart(grouped_selp_deg).mark_bar(width=5,  opacity=0.7, color="black").encode(
distrib = alt.Chart(grouped_selp_deg).mark_bar(width=5,  opacity=0.7).encode(
        x=alt.X('degree:Q', title="k", axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domainMin=0)),
        y=alt.Y('Percentage:Q'),
        # facet=alt.Facet("q:O")
        # color="q"
    )

In [ ]:
# alt.layer(pdf, distrib, data=grouped_selp_deg).facet(column='q')
alt.layer(pdf, distrib, data=grouped_selp_deg).facet(
    facet=alt.Facet(
        'q / p',
        header=alt.Header(labelFontSize=18, titleFontSize=20)
    ),
    columns=2
).configure_axisY(
    titleAngle=0,
    titleX=-80,
)


# alt.hconcat(freq_table, background + data_plot, spacing=100).properties(padding=100)

## Hsize

### Group by

In [ ]:
df_hsizes = df_hsizes[df_hsizes["N_nodes"] == 1000]

In [ ]:
df_fits2 = df_fits2.round({'q_frac': 3})

In [ ]:
grouped = df_hsizes.groupby('q / p')['hsize'].value_counts(normalize=True).reset_index(name='Percentage')
grouped

In [ ]:
grouped = grouped.round({'q / p': 2})

In [ ]:
grouped['binomial'] = 0  # Initialize the column

for q in qs:
    q = round(q, 2)
    # df = df_hsizes[df_hsizes["q"] == f"{q}p"]
    df = df_hsizes[df_hsizes["q"] == q]
    
    # fit_values = df_fits2[(df_fits2["q_frac"] == f"{q}p") & (df_fits2["measure"] == "hsize") & (df_fits2["bounds"] == "p Fixed")]
    fit_values = df_fits2[(df_fits2["q_frac"] == q) & (df_fits2["measure"] == "hsize") & (df_fits2["bounds"] == "p Fixed")]

    # p = 0.03
    # x_values = range(60)

    # p = 0.1
    x_values = range(200)
    
    for x in x_values:
        binom_pdf = stats.binom.pmf(x, fit_values.Neff, fit_values.peff)
        grouped.loc[(grouped['hsize'] == x) & (grouped['q / p'] == q), 'binomial'] = binom_pdf

In [ ]:
# grouped_selp = grouped[grouped.q.isin(["0.0p", "0.3p", "0.7p", "1.0p"])]
grouped_selp = grouped[grouped["q / p"].isin([0, 0.3, 0.7, 1])]

In [ ]:
grouped_selp

In [ ]:
# pdf = alt.Chart(grouped_selp).mark_line(width=1, opacity=0.7, color="darkgrey").encode(
pdf = alt.Chart(grouped_selp).mark_line(width=1, opacity=0.7, color="black", fill=None).encode(
        # x=alt.X('hsize:O', title="Hyperedge Size", axis=alt.Axis(tickMinStep=10), scale=alt.Scale(domain=list(range(3, 51)), padding=0.1, paddingInner=0.1)),
        x=alt.X('hsize:Q', title="m", axis=alt.Axis(tickMinStep=5)),
        y=alt.Y('binomial:Q', title="P(m)"),
        # color="q"
        # facet=alt.Facet("q:O")    
)

In [ ]:
# distrib = alt.Chart(grouped_selp).mark_bar(width=5,  opacity=0.7, color="black").encode(
distrib = alt.Chart(grouped_selp).mark_bar(width=5,  opacity=0.7).encode(
        # x=alt.X('hsize:O', title="Hyperedge Size", axis=alt.Axis(tickMinStep=10), scale=alt.Scale(domain=list(range(3, 51)), padding=0.1, paddingInner=0.1)),
        x=alt.X('hsize:Q', title="m", axis=alt.Axis(tickMinStep=5)),
        y=alt.Y('Percentage:Q'),
        # facet=alt.Facet("q:O")
        # color="q"
    )

In [ ]:
alt.layer(pdf, distrib, data=grouped_selp).facet(
    facet=alt.Facet('q / p', header=alt.Header(labelFontSize=18, titleFontSize=20)),  columns=2
).configure_axisY(
    titleAngle=0,
    titleX=-80,
)

## Hyperedge composition

In [ ]:
df_fraction2

In [ ]:
df_fraction2['strat'] = df_fraction2['strat'].replace(['frequent'], ['majority'])

In [ ]:
df_fraction2 = df_fraction2.round({"q / p": 3})

In [ ]:
# 1000 nodes
df_fraction_1000 = df_fraction2[df_fraction2["N / E"] == 5]

In [ ]:
df_test = df_fraction2[(df_fraction2["N / E"].isin([10])) & (df_fraction2["strat"] == "max")]

In [ ]:
error = alt.Chart(df_fraction_1000).mark_errorband(extent="stdev").encode(
    x="q / p",
    y=alt.Y(
        "gini:Q",
        title="gini"
    ),
)

mean_line = alt.Chart(df_fraction_1000).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean(gini)",
        title="gini"
    ),
)

error + mean_line

In [ ]:
# Calculate mean and standard deviation for each category
summary_data = df_test.groupby('q / p').agg({'gini': ['mean', 'std']}).reset_index()
summary_data.columns = ['q / p', 'Mean', 'Std']# Calculate mean and standard deviation for each category

In [ ]:
df_split = pd.DataFrame(columns=["q / p", "value", "type"])

for i, row in summary_data.iterrows():

    df_split = pd.concat([df_split, pd.DataFrame({'q / p': [row["q / p"]], 'value': [row.Mean], 'type': ["Ḡ"]})], ignore_index=True)
    df_split = pd.concat([df_split, pd.DataFrame({'q / p': [row["q / p"]], 'value': [row.Std], 'type': ["σ"]})], ignore_index=True)
    
    # df_split = df_split.append({'q / p': row["q / p"], 'value': row.Mean, 'type': "Mean"}, ignore_index=True)
    # df_split = df_split.append({'q / p': row["q / p"], 'value': row.Std, 'type': "Std"}, ignore_index=True)

In [ ]:
df_split.head()

In [ ]:
# error = alt.Chart(df_test).mark_errorband(extent="stdev").encode(
#     x="q / p",
#     y=alt.Y(
#         "gini:Q",
#         title="gini"
#     ),
# )
# std = alt.Chart(df_test).mark_line().encode(
#     x="q / p",
#     y=alt.Y(
#         "stdev(gini)",
#         title="gini"
#     ),
# )

# mean_line = alt.Chart(df_test).mark_line().encode(
#     x="q / p",
#     y=alt.Y(
#         "mean(gini)",
#         title="gini"
#     ),
# )

# mean_line + std

alt.Chart(df_split).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "value",
        title="G",
    ),
        color=alt.Color("type", scale=alt.Scale(range=["black", markColor]),
                       legend=alt.Legend(title="", orient="top-right", symbolSize=750, symbolStrokeWidth=3))
).configure_legend(
titleFontSize=20,
labelFontSize=16,
).properties(height=330).configure_axisY(
    titleAngle=0,
    titleX=-60,
)



In [ ]:
# alt.layer(pdf, distrib, data=grouped_selp_deg).facet(column='q')
alt.layer(error, mean_line, data=df_fraction_1000).facet(column='strat')

In [ ]:
alt.Chart(df_fraction2).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean(gini)",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        title="mean(Gini*)"
    ),
    # color=alt.Color("N / E:N", scale=alt.Scale(scheme="category10"))
    # color=alt.Color("N / E:N", scale=alt.Scale()),
    strokeDash="N / E"
).facet(column='strat')

In [ ]:
# frequent => mettre majority
# Augmenter la taille des labels, axis, 

gini_strat = alt.Chart(df_fraction_1000).mark_line(color=markColor, opacity=1, size=3).encode(
    x="q / p",
    y=alt.Y(
        "mean(gini)",
        # title="normalized gini",
        title="Ḡ"
    ),
    # color=alt.Color("N / E:N", scale=alt.Scale()),
    strokeDash=alt.StrokeDash("strat", scale=alt.Scale(range=[[1,0], [4, 1], [8, 2], [4, 8]]), legend=alt.Legend(title="Strategy", orient="top-right", symbolSize=750, symbolStrokeWidth=3)),
).configure_legend(
titleFontSize=20,
labelFontSize=16,
).properties(height=330).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

gini_strat

Valeur de G pour q / p =1

In [ ]:
def sqrtApply(v):
    return 1 / math.sqrt(v)

df_fraction2["sqrt"] = df_fraction2["N / E"].apply(sqrtApply)

In [ ]:
c1 = alt.Chart(df_fraction2[df_fraction2["q / p"] == 1]).mark_line(color=markColor, opacity=1, size=3).encode(
    x=alt.X("sqrt", title="1 / sqrt(N / E)"),
    y=alt.Y(
        "mean(gini)",
        # title="normalized gini",
        title="Ḡ"
    )
)

c2 = alt.Chart(df_fraction2[df_fraction2["q / p"] == 1]).mark_point(
    size=80, opacity=0.8, fill=markColor, stroke="black", shape="diamond").encode(
    x=alt.X("sqrt", title="1 / sqrt(N / E)"),
    y=alt.Y(
        "mean(gini)",
        # title="normalized gini",
        title="Ḡ"
    )
)


alt.layer(c1, c2).configure_legend(
titleFontSize=20,
labelFontSize=16,
).properties(height=330).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
# mettre la legende à l'interieur
# Utiliser couleur et stroke?
gini_size = alt.Chart(df_fraction2[df_fraction2.strat == "weighted"]).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean(gini)",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        # title="normalized gini",
        title="Ḡ"
    ),
    # color=alt.Color("N / E:N", scale=alt.Scale(scheme="category10"))
    color=alt.Color("N / E:O", scale=alt.Scale(scheme="blues"), legend=alt.Legend(orient="top-right")),
    # strokeDash="strat"
).configure_legend(
titleFontSize=20,
labelFontSize=16,
).properties(height=330).configure_axisY(
    titleAngle=0,
    titleX=-60,
)


gini_size

In [ ]:
# Weird legend position
# alt.hconcat(gini_strat, gini_size)

## Normalize 

In [ ]:
df_fraction2_var = (df_fraction2
    # .groupby('q / p')
    .groupby(['q / p', "N / E", "strat"])
    # ['value']
    ['gini']
    .agg(['mean', 'std'])
    .assign(stdNorm = lambda df: df['std'] / df['mean'])
    .reset_index())

In [ ]:
df_fraction2_var[(df_fraction2_var.strat == "max") & (df_fraction2_var["N / E"] == 10)]

In [ ]:
# Figure 6. Essayer en couleurs. Enlever le strat. Double colones
# Pour le max de strat max: metrre plus de point
# Tracer en fct de E / N en faisant varier N
alt.Chart(df_fraction2_var).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "stdNorm",
        # "std",
        title="δ"
    ),
    color=alt.Color("N / E:O", scale=alt.Scale(scheme="blues")),
    # strokeDash=alt.StrokeDash("N / E:O", legend=alt.Legend(symbolSize=750))
).facet(
    facet=alt.Facet("strat", header=alt.Header(title="Strategy", labelFontSize=18, titleFontSize=20))
       ).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
alt.Chart(df_fraction2_var).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "std",
        # "std",
        title="σ"
    ),
    color=alt.Color("N / E:O", scale=alt.Scale(scheme="blues")),
    # strokeDash=alt.StrokeDash("N / E:O", legend=alt.Legend(symbolSize=750))
).facet(
    facet=alt.Facet("strat", header=alt.Header(title="Strategy", labelFontSize=18, titleFontSize=20))
       ).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
alt.Chart(df_fraction2_var).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean",
        # "std",
        title="mean"
    ),
    color=alt.Color("N / E:O", scale=alt.Scale(scheme="blues")),
    # strokeDash=alt.StrokeDash("N / E:O", legend=alt.Legend(symbolSize=750))
).facet(
    facet=alt.Facet("strat", header=alt.Header(title="Strategy", labelFontSize=18, titleFontSize=20))
       ).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
# df_fraction2_max = (df_fraction2_var
#     .groupby(["N / E", "strat"])
#     ['stdNorm']
#     .agg(['max'])
#     # .assign(max = lambda df: df['std'] / df['mean'])
#     .reset_index())

In [ ]:
# # Tracer en fct de E / N en faisant varier N
# alt.Chart(df_fraction2_max).mark_line().encode(
#     x="N / E",
#     y=alt.Y(
#         "max",
#         # title="Hyperedge Purity"
#         # title="max(n1, n2) / n1 + n2"
#         title="gini*",
#     )
# ).facet("strat")

# Hyperedge Gini Distribution

In [ ]:
df_fraction2[(df_fraction2["q / p"].isin([0.2])) & (df_fraction2["N / E"].isin([10]))]

In [ ]:
alt.Chart(df_fraction2[(df_fraction2["q / p"].isin([0.2])) & (df_fraction2["N / E"].isin([10, 1, 2.5]))]).transform_density(
    'gini',
    groupby=['N / E', "strat"],
    as_=['value', 'density'],
).mark_line().encode(
    x=alt.X("value:Q", title="G"),
    y=alt.Y('density:Q', title="P(G)"),
    color=alt.Color("N / E:O", scale=alt.Scale(scheme="blues"))
).facet(facet=alt.Facet("strat", header=alt.Header(title="Strategy", labelFontSize=18, titleFontSize=20))
       ).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
alt.Chart(df_fraction2[(df_fraction2["q / p"].isin([0.05, 0.1, 0.2, 0.3, 0.4])) & (df_fraction2["N / E"].isin([10])) & (df_fraction2["strat"] == "max")]).transform_density(
    'gini',
    groupby=['q / p'],
    as_=['value', 'density'],
).mark_line().encode(
    x=alt.X("value:Q", title="G"),
    y=alt.Y('density:Q', title="P(G)"),
    color=alt.Color("q / p:N", scale=alt.Scale(scheme="purples",  domainMax=1, domainMin=0.1))
).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
# gini_plot = alt.Chart(df_fraction2[(df_fraction2.N_nodes == 1000) & (~df_fraction2["q / p"].isin([0, 0.01]))]).transform_density(
# gini_plot = alt.Chart(df_fraction2[(df_fraction2["N / E"] == 5) & (~df_fraction2["q / p"].isin([0, 0.01]))]).transform_density(
gini_plot = alt.Chart(df_fraction2[(df_fraction2["N / E"] == 10) & (~df_fraction2["q / p"].isin([0]))]).transform_density(
    'gini',
    groupby=['q / p', "strat"],
    as_=['gini', 'density'],
    # counts=True
).mark_line().encode(
    x=alt.X("gini:Q", title="G"),
    # y=alt.Y('density:Q', title="P(G)", scale=alt.Scale(domainMax=8)),
    y=alt.Y('density:Q', title="P(G)", scale=alt.Scale()),
    color=alt.Color("q / p:N",  scale=alt.Scale(scheme="purples"))
)

In [ ]:
gini_plot.facet(facet=alt.Facet("strat", header=alt.Header(title="Strategy", labelFontSize=18, titleFontSize=20))).configure_axisY(
    titleAngle=0,
    titleX=-60,
)

In [ ]:
gini_plot = alt.Chart(df_fraction2["q / p"].isin([0.2])).transform_density(
    'gini',
    groupby=['q / p', "strat"],
    as_=['gini', 'density'],
    counts=True
).mark_line().encode(
    x=alt.X("gini:Q", title="gini"),
    y='density:Q',
    color=alt.Color("strat",  scale=alt.Scale(scheme="greys"))
)

# Distrib effectives

In [ ]:
df_fits

## Degree distrib fit

In [ ]:
points = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff', title="p_eff"),
).properties(
    width=800,
    height=300
)

In [ ]:
regresssion = points.transform_regression('q', 'peff', method="poly").mark_line(color="black")

In [ ]:
line = alt.Chart(pd.DataFrame({'y': [p]})).mark_rule().encode(y='y')

In [ ]:
# points + regresssion + line
# Pour plusieurs valeur de N
points + line

### Normalize

In [ ]:
df_fits["peff / p"] = df_fits["peff"] / df_fits["p"] 
df_fits["p_h_eff / p"] = df_fits["p_hsize_eff"] / df_fits["p"] 
df_fits["q / p"] = df_fits["q"] / df_fits["p"] 

df_fits["Neff / Nedges"] = df_fits["Neff"] / N_edges
df_fits["NeffHsize / N"] = df_fits["n_eff_hsize"] / N

In [ ]:
df_fits2["q / p"] = df_fits2["q"] / df_fits2["p"] 
df_fits2["peff / p"] = df_fits2["peff"] / df_fits2["p"] 
df_fits2["neff / Nedges"] = df_fits2["Neff"] / N_edges
df_fits2["neff / N"] = df_fits2["Neff"] / N

## Old

In [ ]:
points = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff / p'),
).properties(
    width=800,
    height=300
)

points

In [ ]:
points_Neff = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('Neff / Nedges'),
).properties(
    width=800,
    height=300
)

points_Neff

In [ ]:
alt.Chart(df_fits).mark_line().encode(
    x=alt.X('peff / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('Neff / Nedges'),
).properties(
    width=800,
    height=300
)

In [ ]:
alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('Neffpfixed'),
).properties(
    width=800,
    height=300
)

In [ ]:
# Fitter par une droite (rouge)
# Point plus grand,

alt.Chart(df_fits2[df_fits2.measure == "degree"]).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / Nedges'),
    color="bounds"
).properties(
    width=800,
    height=300
)

In [ ]:
alt.Chart(df_fits2[df_fits2.measure == "degree"]).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff / p'),
    color="bounds"
).properties(
    width=800,
    height=300
)

In [ ]:
df_fits2

## Hsize fits

In [ ]:
points = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('p_h_eff / p'),
).properties(
    width=800,
    height=300
)

points

In [ ]:
points_Neff = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y("NeffHsize / N"),
).properties(
    width=800,
    height=300
)

points_Neff

## Several bounds

In [ ]:
chart = alt.Chart(df_fits2[df_fits2.measure == "hsize"]).mark_point().encode(
    x=alt.X('q / p'),
    # x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / N'),
    # color="bounds"
).properties(
    width=800,
    height=300
)

# chart

In [ ]:
chart + chart.transform_regression('q / p', 'neff / N').mark_line()

In [ ]:
alt.Chart(df_fits2[df_fits2.measure == "hsize"]).mark_point().encode(
    x=alt.X('q / p'),
    # x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff / p'),
    color="bounds"
).properties(
    width=800,
    height=300
)

In [ ]:
df = df_fits2[(df_fits2.measure == "hsize") & (df_fits2.bounds == "p Fixed") & (~df_fits2["q / p"].isin([0.01, 0.05]))]
chart = alt.Chart(df).mark_point(size=80, opacity=0.8, fill=markColor, stroke="black").encode(
# chart = alt.Chart(df).mark_point(size=60, opacity=0.7).encode(
    x=alt.X('q / p', axis=alt.Axis(format='')),
    # x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / N', title="N* / N", scale=alt.Scale(domain=[0, 1.1])),
).properties(
    width=600,
    height=300
)

In [ ]:
hsize_chart = chart.transform_regression('q / p', 'neff / N').mark_line(color=markColor) + chart
hsize_chart

In [ ]:
df = df_fits2[(df_fits2.measure == "degree") & (df_fits2.bounds == "p Fixed") & (~df_fits2["q / p"].isin([0.01, 0.05]))]
chart2 = alt.Chart(df).mark_point(size=80, opacity=0.8, fill=markColor, stroke="black", shape="square").encode(
    # x=alt.X('q / p', sort=q_frac_order, axis=None),
    x=alt.X('q / p', axis=alt.Axis(format='')),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / Nedges', title="E* / E", scale=alt.Scale(domain=[0, 1.1])),
).properties(
    width=600,
    height=300
)

In [ ]:
reg_degree = chart2.transform_regression('q / p', 'neff / Nedges', params=False).mark_line(color=markColor)

In [ ]:
degree_chart = reg_degree + chart2 
degree_chart

### Regression coeffs

In [ ]:
import altair_transform

In [ ]:
reg = chart2.transform_regression('q / p', 'neff / Nedges', params=True)
# print(altair_transform.extract_data(reg))

In [ ]:
chart2.transform_regression(
    'q / p', 'neff / Nedges', params=True
).transform_calculate(
    intercept='datum.coef[0]',
    slope='datum.coef[1]',
).mark_text(align='left').encode(
    x=alt.value(20),  # pixels from left
    y=alt.value(20),  # pixels from top
    text='slope:N'
)

In [ ]:
reg2 = chart.transform_regression('q / p', 'neff / N', params=True)
# print(altair_transform.extract_data(reg2))

In [ ]:
chart.transform_regression(
    'q / p', 'neff / N', params=True
).transform_calculate(
    intercept='datum.coef[0]',
    slope='datum.coef[1]',
).mark_text(align='left').encode(
    x=alt.value(20),  # pixels from left
    y=alt.value(20),  # pixels from top
    text='intercept:N'
)

In [ ]:
# Fixer p est plus facilement explicable: plutot parler de ca dans le papier

In [ ]:
%%time
a = ["dw"] * 400000000

In [ ]:
%%time
a = ["dw" for i in range(400000000)] 